In [ ]:
from monai.transforms import (
	EnsureChannelFirstd,
	LoadImaged,
	Orientationd,
	Spacingd,
	SaveImaged,
	Resized,
)
from monai.data import MetaTensor

import numpy as np
import os
import nibabel as nib

# 1) Ubah variabel global berikut sesuai struktur direktori dan keinginan

In [ ]:
# Root directory data
data_dir = "raw_data"

# Root directory output
output_dir = "resized_data"

# Dimensi citra tujuan
new_dimension = (256,256,256)

# 2) Jalankan code berikut

In [ ]:
def get_affine(original_affine, original_shape, new_dimension):
	original_spacing = np.diag(original_affine)[:3]
	new_spacing = (original_spacing[0] * original_shape[0] / new_dimension[0],
					original_spacing[1] * original_shape[1] / new_dimension[1],
					original_spacing[2] * original_shape[2] / new_dimension[2])
	new_affine = np.copy(original_affine)
	new_affine[0, 0] = new_spacing[0]
	new_affine[1, 1] = new_spacing[1]
	new_affine[2, 2] = new_spacing[2]
	return new_affine

def get_new_spacing(img_path, new_dimension=[256,256,256]):
	nifti_file = nib.load(img_path)
	affine = nifti_file.affine
	voxel_spacing = nib.affines.voxel_sizes(affine)
	image_dimensions = list(nifti_file.get_fdata().shape)
	total_size = [voxel_spacing[0] * image_dimensions[0], voxel_spacing[1] * image_dimensions[1], voxel_spacing[2] * image_dimensions[2]]
	new_spacing = (total_size[0] / new_dimension[0], total_size[1] / new_dimension[1], total_size[2] / new_dimension[2])
	return new_spacing

In [ ]:
os.makedirs(output_dir, exist_ok=True)

In [ ]:
loader = LoadImaged(keys=("label"), image_only=False)
ensure_channel_first = EnsureChannelFirstd(keys=["label"])
orientation = Orientationd(keys=["label"], axcodes="PLI")
resize = Resized(keys=["label"], spatial_size=new_dimension, mode="nearest-exact")

for dir in os.listdir(data_dir):
	if os.path.exists(os.path.join(output_dir, dir)):
		continue
	dir_path = os.path.join(data_dir, dir)
	label_path = os.path.join(dir_path, "aneurysms.nii.gz")
	train_dict = {'label': label_path}

	spacing = Spacingd(keys=["label"], pixdim=get_new_spacing(label_path, list(new_dimension)), mode=("nearest"))

	data_dict = loader(train_dict)
	data_dict = ensure_channel_first(data_dict)
	data_dict = orientation(data_dict)
	data_dict = spacing(data_dict)
	data_dict = resize(data_dict)

	os.makedirs(os.path.join(output_dir, dir), exist_ok=True)
	saver = SaveImaged(keys=["label"], output_dir=os.path.join(output_dir, dir), output_postfix="", separate_folder=False)
	output_label_path = f"{dir}_aneurysms.nii"
	data_dict["label"].meta["filename_or_obj"] = output_label_path
	saver(data_dict)

In [ ]:
loader = LoadImaged(keys=("image"), image_only=False)
ensure_channel_first = EnsureChannelFirstd(keys=["image"])
orientation = Orientationd(keys=["image"], axcodes="PLI")
resize = Resized(keys=["image"], spatial_size=new_dimension, mode="trilinear")

for dir in os.listdir(data_dir):
	dir_path = os.path.join(data_dir, dir)
	img2_path = os.path.join(dir_path, "pre", "TOF.nii.gz")
	train_dict = {'image': img2_path}

	spacing = Spacingd(keys=["image"], pixdim=get_new_spacing(img2_path, list(new_dimension)), mode=("bilinear"))

	data_dict = loader(train_dict)
	data_dict = ensure_channel_first(data_dict)
	data_dict = orientation(data_dict)
	data_dict = spacing(data_dict)
	data_dict = resize(data_dict)

	img1_new_affine = get_affine(data_dict['image'].affine, data_dict['image'].shape[1:], new_dimension)
	data_dict['image'].affine = img1_new_affine
	data_dict['image'].affine = img1_new_affine

	os.makedirs(os.path.join(output_dir, dir), exist_ok=True)
	saver = SaveImaged(keys=["image"], output_dir=os.path.join(output_dir, dir), output_postfix="", separate_folder=False)
	output_image1_path = f"{dir}_TOF-pre.nii"
	data_dict["image"].meta["filename_or_obj"] = output_image1_path
	saver(data_dict)

In [ ]:
loader = LoadImaged(keys=["image"], image_only=False)
ensure_channel_first = EnsureChannelFirstd(keys=["image"])
orientation = Orientationd(keys=["image"], axcodes="PLI")
resize = Resized(keys=["image"], spatial_size=new_dimension, mode="trilinear")

for dir_name in os.listdir(data_dir):
	dir_path = os.path.join(data_dir, dir_name)
	img2_path = os.path.join(dir_path, "pre", "struct_aligned.nii.gz")
	if not os.path.exists(img2_path):
		print(f"Skipping {dir_name}, file not found.")
		continue

	train_dict = {'image': img2_path}
	spacing = Spacingd(keys=["image"], pixdim=get_new_spacing(img2_path, list(new_dimension)), mode="bilinear")

	data_dict = loader(train_dict)
	data_dict = ensure_channel_first(data_dict)
	data_dict = orientation(data_dict)
	data_dict = spacing(data_dict)
	data_dict = resize(data_dict)

	img = data_dict['image']
	img_new_affine = get_affine(img.affine, img.shape[1:], new_dimension)
	img.affine = img_new_affine

	img_data = img[0] if isinstance(img, MetaTensor) else img
	data = img_data.numpy()
	min_data, max_data = np.min(data), np.max(data)
	normalized_data = (data - min_data) / (max_data - min_data + 1e-5) * 65535
	uint16_data = np.round(normalized_data).astype(np.uint16)

	original_nib = nib.load(img2_path)
	final_nifti = nib.Nifti1Image(uint16_data, img.affine, original_nib.header)
	final_nifti.header['datatype'] = 512
	final_nifti.header['bitpix'] = 16

	output_subject_dir = os.path.join(output_dir, dir_name)
	os.makedirs(output_subject_dir, exist_ok=True)
	final_path = os.path.join(output_subject_dir, f"{dir}_struct_init.nii")
	nib.save(final_nifti, final_path)

	print(f"Saved: {final_path}")
	break


# 3) Struktur akhir directory data resize

resized_data/  
|-- 10001/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10001_aneurysms.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10001_TOF-pre.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10001_struct_init.nii  
|-- 10002/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10002_aneurysms.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10002_TOF-pre.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10002_struct_init.nii  
|-- 10003/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10003_aneurysms.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10003_TOF-pre.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10003_struct_init.nii  
...